### This program solves the model of Aiyagari(94)

In [1]:
# Importing packages
import numpy as np

# needed for compact printing of numpy arrays
# use precision to set the number of decimal digits to display
# use suppress=True to show values in full decimals instead of using scientific notation
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf)

#### The function markov_approx is copied below.

In [2]:
# markovapprox: approximates a continuous AR(1) process with a Markov chain
# Eva Carceles-Poveda's version. You can use this function, Floden's tauchen function, or Sargent's tauchen function.

from scipy.stats import norm
def markov_approx(rho, sigma, m, N):
    """ Syntax: [Tran,s,p,arho,asigma]=markovapprox(rho,sigma,m,N)
    
    This function approximates a first-order autoregressive process 
    with persistence rho and innovation standard deviation sigma with 
    an N state Markov chain; m determines the width of the discretized 
    state space, Tauchen uses m=3, with ymax=m*vary,ymin=-m*vary, where 
    ymax and ymin are the two boundary points, Tran is the transition 
    matrix of the Markov chain, s is the discretized state space, p is 
    the chain stationary distribution, arho is the theoretical first 
    order autoregression coefficient for the Markov chain, asigma is 
    the theoretical standard deviation for the Markov chain.
    
    Translated from Eva Carceles-Poveda 2003 MATLAB code
    """
    
    # Discretize the state space
    stvy = np.sqrt(sigma**2/(1-rho**2))   # standard deviation of y(t)
    ymax = m*stvy                         # upper boundary of state space
    ymin = -ymax                          # lower boundary of state space
    w = (ymax-ymin)/(N-1)                 # distance between points
    s = w * np.arange(ymin/w, ymax/w+1)   # the discretized state space        
    
    
    # Calculate the transition matrix
    Tran = np.zeros((N,N))
    for j in np.arange(0,N):
        for k in np.arange(1,N-1):
            Tran[j,k] = norm.cdf(s[k]-rho*s[j]+w/2,0,sigma) - norm.cdf(s[k]-rho*s[j]-w/2,0,sigma);
            
        Tran[j,0] = norm.cdf(s[0]-rho*s[j]+w/2,0,sigma);
        Tran[j,N-1] = 1 - norm.cdf(s[N-1]-rho*s[j]-w/2,0,sigma);
        
    # Check that Tran is well specified
    if not np.all(np.isclose(np.sum(Tran.T, axis=0), np.squeeze(np.ones((1,N))))):
        # find rows not adding up to one
        str = (np.absolute(np.sum(Tran.T, axis=0))-np.squeeze(np.ones((1,N)))<1e-14).nonzero()          
        print('error in transition matrix')
        print('rows', str[0],' do not sum to one')
    
    
    # Calculate the invariant distribution of Markov chain
    Trans = Tran.T
    p = (1/N)*np.ones((N,1)) # initial distribution of states
    test = 1;
    while test > 1e-8:
        p1 = np.matmul(Trans,p)
        test = np.max(np.abs(p1-p))
        p = p1
    
    
    meanm = np.matmul(s,p)            # mean of invariant distribution of chain
    varm = np.matmul((s-meanm)**2,p)  #variance of invariant distribution of chain  
    midaut1 = np.matmul((s-meanm)[:, np.newaxis],(s-meanm)[np.newaxis, :]) # cross product of deviation from mean of yt and yt-1                    
    probmat = np.matmul(p,np.ones((1,N)))     # each column is invariant distribution   
    midaut2 = Tran*probmat*midaut1 # product of the first two terms is joint distribution of (Yt-1,Yt)                                    
    autcov1 = np.sum(midaut2)    #  first-order auto-covariance
    
    
    arho = autcov1/varm           # theoretical first order autoregression coefficient
    asigma = np.sqrt(varm)           # theoretical standard deviation
    
    return Tran, s, p, arho, asigma

In [3]:
# Parameter values
beta = 0.96
timepref = 1/beta-1
mu = 3
theta = 0.36
delta = 0.08
rho = 0.9
sigma = 0.4
sigmain = sigma*np.sqrt(1-rho**2)

In [4]:
# N state markov chain
N = 7

Pi,logs,invdist = markov_approx(rho,sigmain,3,N)[0:3] # Pi gives Pij=prob(j given i)
s = np.exp(logs)
labor = s@invdist
lt = max(s.shape)
Pii = Pi.T # Pij=prob(i given j)

In [5]:
# Algorithm parameters
tolv = 1e-07
tolr = 1e-04
tola = 1e-03

ini = 1
if ini:
    r = 0.04
else:
    r = 0.02298754441505 #394

# Initial interest rate and grid for the states
k = ((r+delta)/(theta*labor**(1-theta)))**(1/(theta-1))
w = (1-theta)*k**theta*labor**(-theta)

# asset limit and gridb=0;
b = 0
if r<=0:
    phi = b
else:
    phi  = min(b, w*s[0]/r)

minkap = -phi
maxkap = 16.00
inc = 0.2
#inc = 0.405
kgrid  = np.arange(minkap,maxkap+inc,inc)
lk = max(kgrid.shape)

# Main loop
testr = 1
testa = 1
iter1 = 0
while testr > tolr or testa > tola:
    iter1 = iter1+1
    print("Iteration", iter1, ":")
    oldr = r
    
    k = ((r+delta)/(theta*labor**(1-theta)))**(1/(theta-1))
    w = (1-theta)*k**theta*labor**(-theta)
    
    # iterate on the value function and compute the optimal policy
    c = np.zeros((lk*lk,lt))
    u = np.zeros((lk*lk,lt))
    for t in np.arange(0,lt):
        for i in np.arange(0,lk):
            for j in np.arange(0,lk):
                c[i*lk+j,t] = (1+r)*kgrid[i]+w*s[t]-kgrid[j]
                if c[i*lk+j,t]<0:
                    c[i*lk+j,t] = 1e-07
    
    if mu==1:
        u = np.log(c)
    else:
        u = (c**(1-mu)-1)/(1-mu)
    
    
    V0 = np.ones((lk,lt))
    V1 = np.zeros((lk,lt))
    optim = np.zeros((lk,lt),dtype=int)
    
    # Value function iteration
    iter2 = 0
    while np.linalg.norm(V1-V0) > tolv:
        #print(iter2, np.linalg.norm(V0-V1))
        iter2 = iter2+1
        V0 = V1.copy()
        for j in np.arange(0,lt):
            for i in np.arange(0,lk):
                f = u[i*lk:(i+1)*lk,j]+np.matmul(beta*V0,Pii[:,j])
                V1[i,j] = f.max()
                optim[i,j] = f.argmax()
                

    # Policy function conditional on shock
    polk = kgrid[optim]   
    
    # Calculate the invariant distribution
    gmat = np.zeros((lk,lk,lt))
    trans = np.zeros((lk*lt,lk*lt))
    for j in np.arange(0,lt):
        for i in np.arange(0,lk):
            gmat[i,optim[i,j],j] = 1;

        trans[j*lk:(j+1)*lk,:] = np.kron(Pi[j,:],gmat[:,:,j])
    
    trans = trans.T
    probst = (1/(lt*lk))*np.ones((lt*lk,1))
    
    test = 1
    while test > 10**(-5):
        probst1 = trans@probst
        test = np.abs(probst1-probst).max()
        probst = probst1.copy()
    
    
    # change the dimension of polk
    kk = polk.flatten(order='F')
    meank = probst.T@kk
    
    # Update r if necessary
    rstar=theta*((meank)**(theta-1))@(labor**(1-theta))-delta
    testr = np.abs(r-rstar)
    testa = np.abs(k-meank)
    
    if iter1==1:
        if rstar>(1/beta)-1:
            rstar=(1/beta)-1
        
        rhigh=max([r,rstar]);
        rlow=min([r,rstar]);
    elif meank > k:
        print('saving too much (meank>k), so reducing r!')
        rhigh=r
    else:
        print('saving too little (meank<k), so increasing r!')
        rlow=r
    
    r=rhigh*0.5+rlow*0.5
    print('testr testa oldr rstar')
    print(np.around([testr, testa[0], oldr, rstar],4))

print('Final solution: r, k, meank, s, testr testa')
print(np.around([r, k[0], meank[0], (delta*meank/((meank**theta)*labor**(1-theta)))[0], testr, testa[0]],4))

Iteration 1 :
testr testa oldr rstar
[ 0.0407  5.6457  0.04   -0.0007]
Iteration 2 :
saving too little (meank<k), so increasing r!
testr testa oldr rstar
[0.008  0.9404 0.0197 0.0276]
Iteration 3 :
saving too much (meank>k), so reducing r!
testr testa oldr rstar
[0.0161 2.0042 0.0298 0.0137]
Iteration 4 :
saving too much (meank>k), so reducing r!
testr testa oldr rstar
[0.0051 0.6273 0.0247 0.0196]
Iteration 5 :
saving too little (meank<k), so increasing r!
testr testa oldr rstar
[0.0008 0.101  0.0222 0.023 ]
Iteration 6 :
saving too much (meank>k), so reducing r!
testr testa oldr rstar
[0.0023 0.2765 0.0235 0.0212]
Iteration 7 :
saving too much (meank>k), so reducing r!
testr testa oldr rstar
[0.0007 0.0852 0.0228 0.0221]
Iteration 8 :
saving too little (meank<k), so increasing r!
testr testa oldr rstar
[0.0001 0.0086 0.0225 0.0226]
Iteration 9 :
saving too much (meank>k), so reducing r!
testr testa oldr rstar
[0.0003 0.0423 0.0227 0.0223]
Iteration 10 :
saving too much (meank>k), so 

In [6]:
#!jupyter nbconvert --to script AIYAstatic.ipynb